# **Import Libraries**

In [1]:
import warnings

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import FashionMNIST

from tqdm import tqdm

# **Ignoring Any Warning Messages**

In [2]:
warnings.filterwarnings('ignore')

# **Define Important Hyperparameters**

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 0.0005
num_epochs = 100
batch_size = 256
img_dim = 64
z_dim = 256
img_channels = 1
features = 64
fixed_noise = torch.rand(32, z_dim, 1, 1).to(device)

# **Create Discriminator**

In [4]:
class Discriminator(nn.Module):
  def __init__(self, img_channels, features):
    super().__init__()

    self.net = nn.Sequential(
        nn.Conv2d(img_channels, features * 1, 4, 2, 1),
        nn.LeakyReLU(.2),

        nn.Conv2d(features * 1, features * 2, 4, 2, 1),
        nn.BatchNorm2d(features * 2),
        nn.LeakyReLU(.2),

        nn.Conv2d(features * 2, features * 4, 4, 2, 1),
        nn.BatchNorm2d(features * 4),
        nn.LeakyReLU(.2),

        nn.Conv2d(features * 4, features * 8, 4, 2, 1),
        nn.BatchNorm2d(features * 8),
        nn.LeakyReLU(.2),

        nn.Conv2d(features * 8,  1, 4, 2, 0),
        nn.Sigmoid(),
    )

  def forward(self, x):
    return self.net(x)

# **Create Generator**

In [5]:
class Generator(nn.Module):
  def __init__(self, noise_channels, img_channels, features):
    super().__init__()

    self.net = nn.Sequential(
        nn.ConvTranspose2d(noise_channels, features * 16, 4, 1, 0),
        nn.ReLU(),

        nn.ConvTranspose2d(features * 16, features * 8, 4, 2, 1),
        nn.BatchNorm2d(features * 8),
        nn.ReLU(),

        nn.ConvTranspose2d(features * 8, features * 4, 4, 2, 1),
        nn.BatchNorm2d(features * 4),
        nn.ReLU(),

        nn.ConvTranspose2d(features * 4, features * 2, 4, 2, 1),
        nn.BatchNorm2d(features * 2),
        nn.ReLU(),

        nn.ConvTranspose2d(features * 2, img_channels, 4, 2, 1),
        nn.Tanh(),
    )

  def forward(self, x):
    return self.net(x)

# **Define Weights Intialization Function**

In [6]:
def initialize_weights(model):
  for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d,  nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data, 0, .02)

# **Generate Dataloader**

### **Define Transformation**

In [7]:
transforms = transforms.Compose(
    [
        transforms.Resize(img_dim),
        transforms.ToTensor(),
        transforms.Normalize((.5, ), (.5, ))
    ]
)

### **Create Dataloader**

In [8]:
dataloader = DataLoader(FashionMNIST('.', download = True, transform = transforms), batch_size = batch_size, shuffle = True)

# **Training Loop**

In [9]:
gen = Generator(z_dim, img_channels, features).to(device)
initialize_weights(gen)

disc = Discriminator(img_channels, features).to(device)
initialize_weights(disc)

opt_gen  = Adam(gen.parameters(),  lr=lr, betas=(0.5, 0.999))
opt_disc = Adam(disc.parameters(), lr=lr, betas=(0.5, 0.999))

criterion = nn.BCELoss()

for epoch in range(num_epochs):
    gen.train()
    disc.train()

    for real, _ in tqdm(dataloader):
        real = real.to(device)

        noise = torch.randn((batch_size, z_dim, 1, 1)).to(device)
        fake = gen(noise)

        disc_real = disc(real).view(-1)
        disc_fake = disc(fake.detach()).view(-1)

        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        lossD = (lossD_real + lossD_fake) / 2

        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        output = disc(fake)
        lossG = criterion(output, torch.ones_like(output))

        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

    print(f"epoch {epoch+1}: lossD={lossD:.4f}, lossG={lossG:.4f}")

    with torch.no_grad():
        fake = gen(fixed_noise.to(device)).reshape(-1, 1, 64, 64)
        real, _ = next(iter(dataloader))
        real = real.to(device)

        img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)
        img_grid_real = torchvision.utils.make_grid(real[:32], normalize=True)

        torchvision.utils.save_image(img_grid_fake, "fake_grid.png")
        torchvision.utils.save_image(img_grid_real, "real_grid.png")

100%|██████████| 235/235 [02:22<00:00,  1.65it/s]


epoch 1: lossD=0.5676, lossG=1.1875


100%|██████████| 235/235 [02:21<00:00,  1.66it/s]


epoch 2: lossD=0.6074, lossG=1.1143


100%|██████████| 235/235 [02:21<00:00,  1.66it/s]


...

epoch 98: lossD=0.0049, lossG=6.4346


100%|██████████| 235/235 [02:19<00:00,  1.69it/s]


epoch 99: lossD=0.0019, lossG=7.7282


100%|██████████| 235/235 [02:19<00:00,  1.69it/s]


epoch 100: lossD=0.0034, lossG=6.3365
